In [1]:
using DelimitedFiles

In [2]:
function parse_input(filename)
    data = readdlm(filename, '|', String)
    input = join.(sort.(collect.((String.(permutedims(hcat((split.(strip.(data[:, 1], ' '))...))))))))
    output = join.(sort.(collect.((String.(permutedims(hcat((split.(strip.(data[:, 2], ' '))...))))))))
    return input, output
end

parse_input (generic function with 1 method)

# Part 1

In [3]:
function solve_part1(filename)
    output = parse_input(filename)[2]
    output_segment_lengths = permutedims(hcat([length.(o) for o in output]...))
    return count((output_segment_lengths .== 2) .| 
                 (output_segment_lengths .== 4) .| 
                 (output_segment_lengths .== 3) .| 
                 (output_segment_lengths .== 7))
end

println("Test: $(solve_part1("test.txt"))")
println("Solution: $(solve_part1("input.txt"))")

Test: 26
Solution: 548


# Part 2

In [4]:
function identify_digits(digits)
    out_map = Dict([])
    out_map[1] = digits[length.(digits) .== 2][1]
    out_map[4] = digits[length.(digits) .== 4][1]
    out_map[7] = digits[length.(digits) .== 3][1]
    out_map[8] = digits[length.(digits) .== 7][1]
    fives = digits[length.(digits) .== 5]
    sixes = digits[length.(digits) .== 6]
    # 6: length six without full overlap with 7
    out_map[6] = sixes[length.(intersect.(sixes, out_map[7])) .!= 3][1]
    filter!(e->e ≠ out_map[6], sixes) # remove from sixes
    # 9: of remaining length six that has full overlap with 4
    out_map[9] = sixes[length.(intersect.(sixes, out_map[4])) .== 4][1]
    filter!(e->e ≠ out_map[9], sixes) # remove from sixes
    # 0: only length six left
    out_map[0] = sixes[1]
    # 3: only length five with full overlap with 1
    out_map[3] = fives[length.(intersect.(fives, out_map[1])) .== 2][1]
    filter!(e->e ≠ out_map[3], fives) # remove from fives
    # 5: only remaining length five with 3 overlap with 4
    out_map[5] = fives[length.(intersect.(fives, out_map[4])) .== 3][1]
    filter!(e->e ≠ out_map[5], fives)
    # 2: only remaining length 5
    out_map[2] = fives[1]
    # Reverse to code=>value
    return Dict(v=>k for (k,v) in out_map)
end

function decode_output(digit_map, output)
    return parse(Int, join([repr(digit_map[d]) for d in output]))
end

decode_output (generic function with 1 method)

In [5]:
function solve_part2(filename)
    (input, output) = parse_input(filename)
    total = 0
    for i=1:size(input)[1]
        digit_map = identify_digits(input[i, :])
        total += decode_output(digit_map, output[i, :])
    end
    return total
end

println("Test: $(solve_part2("test.txt"))")
println("Solution: $(solve_part2("input.txt"))")

Test: 61229
Solution: 1074888
